# RESEARCH NOTEBOOK --> SmugPlug

In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [4]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "WLD-USDT"
timeframe = "1h"
days = 30

In [5]:
# Get the candles
candles = await clob.get_candles_last_days(
    exchange, trading_pair, timeframe, days, from_trades=False)

2024-11-15 13:34:33,898 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x305577460>
2024-11-15 13:34:33,902 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x30519db40>, 193410.72339325)]']
connector: <aiohttp.connector.TCPConnector object at 0x305577490>


In [8]:
candles.data

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume
timestamp,,,,,,,,,,
2024-10-16 17:00:00,1729098000,2.1981,2.2367,2.194,2.2109,11265107,24950340.3113,80457,5795075,12839866.1785
2024-10-16 18:00:00,1729101600,2.2109,2.2286,2.2027,2.2098,4357107,9650468.6872,44638,1976878,4379848.2665
2024-10-16 19:00:00,1729105200,2.2097,2.2283,2.2066,2.2228,3004938,6661479.9098,32307,1527881,3387468.8614
2024-10-16 20:00:00,1729108800,2.2226,2.2616,2.2211,2.2392,6651074,14915737.8106,52445,3525026,7907757.9259
2024-10-16 21:00:00,1729112400,2.2393,2.2748,2.2388,2.2524,4896431,11058624.1015,42025,2402180,5426694.4294
...,...,...,...,...,...,...,...,...,...,...
2024-11-15 12:00:00,1731672000,2.2866,2.3107,2.2621,2.2711,7612293,17384638.136,78638,3781065,8643394.2549
2024-11-15 13:00:00,1731675600,2.2711,2.292,2.26,2.2761,5511737,12551639.3474,68499,2768643,6306797.1905
2024-11-15 14:00:00,1731679200,2.2762,2.2816,2.2161,2.2243,9874538,22162300.2589,99038,4440224,9968883.4174


In [12]:
candles.plot(type="returns", height=400, width=800)

In [13]:
# EMAs
ema_short = 8
ema_medium = 29
ema_long = 31

# MACD
macd_fast = 22
macd_slow = 36
macd_signal = 17

# ATR
atr_length = 11
atr_multiplier = 1.5

# Add indicators
candles_df = candles.data
candles_df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
candles_df.ta.atr(length=atr_length, append=True)
candles_df.ta.ema(length=ema_short, append=True)
candles_df.ta.ema(length=ema_medium, append=True)
candles_df.ta.ema(length=ema_long, append=True)
candles_df["long_atr_support"] = candles_df["close"].shift(1) - candles_df[f"ATRr_{atr_length}"] * atr_multiplier
candles_df["short_atr_resistance"] = candles_df["close"].shift(1) + candles_df[f"ATRr_{atr_length}"] * atr_multiplier

candles_df.tail(5)

,timestamp,open,high,low,close,volume,quote_asset_volume,n_trades,taker_buy_base_volume,taker_buy_quote_volume,MACD_22_36_17,MACDh_22_36_17,MACDs_22_36_17,ATRr_11,EMA_8,EMA_29,EMA_31,long_atr_support,short_atr_resistance
timestamp,,,,,,,,,,,,,,,,,,,
2024-11-15 12:00:00,1731672000,2.2866,2.3107,2.2621,2.2711,7612293,17384638.136,78638,3781065,8643394.2549,-0.01695571,0.00201542,-0.01897113,0.05534012,2.24996584,2.25972759,2.26218741,2.20358981,2.36961019
2024-11-15 13:00:00,1731675600,2.2711,2.292,2.26,2.2761,5511737,12551639.3474,68499,2768643,6306797.1905,-0.01521515,0.00333865,-0.0185538,0.0532183,2.25577343,2.26081908,2.26305694,2.19127256,2.35092744
2024-11-15 14:00:00,1731679200,2.2762,2.2816,2.2161,2.2243,9874538,22162300.2589,99038,4440224,9968883.4174,-0.01534467,0.00285256,-0.01819723,0.05433481,2.24877934,2.25838448,2.26063464,2.19459778,2.35760222
2024-11-15 15:00:00,1731682800,2.2243,2.2249,2.1646,2.1804,11508381,25204554.6077,106924,5003208,10962419.6541,-0.01682884,0.00121635,-0.01804519,0.0548771,2.23358393,2.25318551,2.25561997,2.14198435,2.30661565
2024-11-15 16:00:00,1731686400,2.1803,2.21,2.161,2.2055,6108794,13356718.6805,53920,3012068,6589488.8037,-0.01720574,0.00074617,-0.01795191,0.05434282,2.22734306,2.25000648,2.25248747,2.09888577,2.26191423


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'MACD'),
                    row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='OHLC'),
              row=1, col=1)

# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
                    row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()

In [15]:
# Generate signal


macdh = candles_df[f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"]
short_ema = candles_df[f"EMA_{ema_short}"]
medium_ema = candles_df[f"EMA_{ema_medium}"]
long_ema = candles_df[f"EMA_{ema_long}"]
close = candles_df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (close > short_ema) & (close > candles_df["long_atr_support"]) & (macdh > 0) 
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (close < short_ema) & (close < candles_df["short_atr_resistance"]) & (macdh < 0)

candles_df["signal"] = 0
candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [16]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='Candlesticks'),
              row=1, col=1)


# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Add ATR support and resistance
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["long_atr_support"],
                         line=dict(color='#00FF00', width=2),
                         name='Long ATR Support'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["short_atr_resistance"],
                         line=dict(color='#FF0000', width=2),
                         name='Short ATR Resistance'), row=1, col=1)

# Add the signal line
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()


# CONCLUSION

In this notebook, we have implemented a strategy combining the MACD (Moving Average Convergence Divergence) indicator with Bollinger Bands. We've visualized these indicators along with the price data and generated signals based on their interactions. This approach provides a solid foundation for our trading strategy.
 
## Key components of our strategy include:
 1. MACD for trend identification
 2. Bollinger Bands for volatility measurement and potential reversal points
 3. A signal line derived from the combination of these indicators
 
 The next step is to backtest this strategy to evaluate its profitability and robustness. For this purpose, we have created a controller file named `macd_bb.py` in this folder. This file implements the logic we've developed here, allowing us to conduct comprehensive backtests in the subsequent notebook.